In [2]:
from neo4j import GraphDatabase

def get_neo4j_driver() -> GraphDatabase.driver:
    """Establishes and returns a Neo4j session for AuraDB."""
    neo4j_uri = 'neo4j+s://9d1381c2.databases.neo4j.io:7687'
    neo4j_user = 'Shirley'
    neo4j_password = 'Sxl19950312'
    return GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Call the function to get the driver
driver = get_neo4j_driver()

# Open a session
with driver.session() as session:
    # Verify connectivity within the session
    session.run("RETURN 1")

print("Connectivity verified successfully.")


from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


# Define a Cypher query to retrieve the required data
query = """
MATCH (n)
RETURN n
"""

# Execute the query and process the results
with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

# Converting Neo4j Data to Pandas DataFrame Format
df = pd.DataFrame(neo4j_data)

#print(df)

Connectivity verified successfully.


In [25]:
with driver.session() as session:
    
    result = session.run("MATCH (p)-[:HAS_RECIPE]->(r:Recipe)-[:HAS_MANUFACTURING_PROCESS]->(i:ManufacturingProcess)"
                         "RETURN p.name AS product, i.Step AS manufacturing_process")
                        

   
    df = pd.DataFrame([record.values() for record in result], columns=result.keys())

    
    product_net = nx.Graph()

    for _, row in df.iterrows():
        product = row['product']
        manufacturing_process = row['manufacturing_process']
        
        
        product_net.add_node(product, manufacturing_process=manufacturing_process)
        
        
        for other_product, data in product_net.nodes(data=True):
            other_manufacturing_process = data.get('manufacturing_process')
            if other_manufacturing_process and other_product != product and manufacturing_process and manufacturing_process in other_manufacturing_process:
              product_net.add_edge(product, other_product, weight=1)



    
    manufacturing_process_partition = community_louvain.best_partition(product_net)

   
    communities = {}
    for node, community in manufacturing_process_partition.items():
        if community not in communities:
            communities[community] = []
        communities[community].append(node)

    
    table_data = []
    for community, products in sorted(communities.items()):
        row = [f"<b>Community {community}</b>"] + ["<br>".join(products)]
        table_data.append(row)

   
    table = go.Table(
        header=dict(values=["Community", "Products"]),
        cells=dict(values=[list(zip(*table_data))[0], list(zip(*table_data))[1]] if table_data else [[]] * 2,  
                   fill_color=[['#f2f2f2', 'white'] * len(table_data)],
                   align=['center', 'left'],
                   font=dict(size=11),
                   height=80)
    )

    
    layout = go.Layout(title="Product Communities Based on Manufacturing Process",
                       width=1000, height=600)

    
    fig = go.Figure(data=[table], layout=layout)
    fig.show()


In [26]:
import pandas as pd
import networkx as nx
import plotly.graph_objs as go
from networkx.algorithms.community import label_propagation_communities

from neo4j import GraphDatabase




with driver.session() as session:
    result = session.run("MATCH (p)-[:HAS_RECIPE]->(r:Recipe)-[:HAS_MANUFACTURING_PROCESS]->(i:ManufacturingProcess)"
                         "RETURN p.name AS product, i.Step AS manufacturing_process")
    
    
    df = pd.DataFrame([record.values() for record in result], columns=result.keys())

# Create teh product net
product_net = nx.Graph()

for _, row in df.iterrows():
    product = row['product']
    manufacturing_process = row['manufacturing_process']
    
    # Add node
    product_net.add_node(product, manufacturing_process=manufacturing_process)

# Detecting Communities Using Label Propagation Algorithms
communities = list(label_propagation_communities(product_net))

# Create the table data
table_data = []
for i, community in enumerate(communities):
    row = [f"<b>Community {i}</b>"] + ["<br>".join(community)]
    table_data.append(row)

# Create the table
table = go.Table(
    header=dict(values=["Community", "Products"]),
    cells=dict(values=[list(zip(*table_data))[0], list(zip(*table_data))[1]] if table_data else [[]] * 2,  
               fill_color=[['#f2f2f2', 'white'] * len(table_data)],
               align=['center', 'left'],
               font=dict(size=11),
               height=80)
)

# Create the layout
layout = go.Layout(title="Product Communities",
                   width=1000, height=600)

# Create the figure
fig = go.Figure(data=[table], layout=layout)
fig.show()


In [27]:
import pandas as pd
import networkx as nx
import plotly.graph_objs as go
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

from neo4j import GraphDatabase


with driver.session() as session:
    result = session.run("MATCH (p)-[:HAS_RECIPE]->(r:Recipe)-[:HAS_MANUFACTURING_PROCESS]->(i:ManufacturingProcess)"
                         "RETURN p.name AS product, i.Step AS manufacturing_process")
    

    df = pd.DataFrame([record.values() for record in result], columns=result.keys())

# Creating feature vectors
# Simple examples are used here, you may need to create more complex feature vectors depending on the situation
X = pd.get_dummies(df['manufacturing_process'])

# Standardised feature vectors
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Clustering using the K-Means algorithm
kmeans = KMeans(n_clusters=5)  # Assuming 5 communities
kmeans.fit(X_scaled)
clusters = kmeans.labels_

# Add clustering results to DataFrame
df['cluster'] = clusters

# Create community lists
communities = {}
for cluster in set(clusters):
    communities[cluster] = df[df['cluster'] == cluster]['product'].tolist()

# Create the data
table_data = []
for cluster, products in sorted(communities.items()):
    row = [f"<b>Cluster {cluster}</b>"] + ["<br>".join(products)]
    table_data.append(row)

# Create the table
table = go.Table(
    header=dict(values=["Cluster", "Products"]),
    cells=dict(values=[list(zip(*table_data))[0], list(zip(*table_data))[1]] if table_data else [[]] * 2,  
               fill_color=[['#f2f2f2', 'white'] * len(table_data)],
               align=['center', 'left'],
               font=dict(size=11),
               height=80)
)

# Create the Layout
layout = go.Layout(title="Product Clusters",
                   width=1000, height=600)

# Create the figure
fig = go.Figure(data=[table], layout=layout)
fig.show()


c:\Users\apple\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\apple\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



In [10]:
from neo4j import GraphDatabase


# Define the manufacturing process community and its corresponding keywords
manufacturing_process_communities = {
        "Mix": ["mix"],
        "Baked": ["bak"],
        "Fried": ["fried"],
        "Pasteurized": ["pasteur"],
        "Canned": ["can"],
        "Dried": ["dry"]
}


def create_manufacturing_process_community_nodes(tx):
    # Create nodes for each manufacturing process community and add the community name as a name attribute to the node
    for community in manufacturing_process_communities.keys():
        tx.run("MERGE (c:ManufacturingProcessCommunity {name: $community})",
               community=community)

def create_product_community_relationships(tx):
    # Create relationships between product and manufacturing process communities
    for community, keywords in manufacturing_process_communities.items():
        query = """
        MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:HAS_MANUFACTURING_PROCESS]->(mp:ManufacturingProcess)
        WHERE """ + " OR ".join(["toLower(mp.Description) CONTAINS '" + keyword + "'" for keyword in keywords]) + """
        WITH p, collect(mp) as processes
        MATCH (c:ManufacturingProcessCommunity {name: $community})
        MERGE (p)-[:MANUFACTURED_BY]->(c)
        """
        tx.run(query, community=community)

with driver.session() as session:
    # Create a manufacturing process community node and add the community name to the node as a name attribute
    session.write_transaction(create_manufacturing_process_community_nodes)
    
    # Create relationships between product and manufacturing process communities
    session.write_transaction(create_product_community_relationships)




C:\Users\apple\AppData\Local\Temp\ipykernel_6040\2200818171.py:33: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_6040\2200818171.py:35: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_manufacturing_process_community_nodes)
C:\Users\apple\AppData\Local\Temp\ipykernel_6040\2200818171.py:38: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_product_community_relationships)
